# Module biogeme.loglikelihood 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the module. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2023-08-04 18:42:03.796249


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.12 [2023-08-04]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import biogeme.logging as blog
logger = blog.get_screen_logger(level=blog.INFO)

In [4]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.loglikelihood as ll
import biogeme.expressions as ex
import biogeme.models as md

This module provides some basic expressions for the contribution of an observation to the (log) likelihood function.

Let's consider first a simple choice model.

In [5]:
V1 = 0
beta = ex.Beta('beta', 0, None, None, 0)
sigma = ex.Beta('sigma', 1, 0, None, 0)
V2 = beta + sigma * ex.bioDraws('V2', 'NORMAL')
V = {1: V1, 2: V2}
condprob = md.logit(V, None, 0)
prob = ex.MonteCarlo(condprob)
print(prob)

MonteCarlo(exp(_bioLogLogitFullChoiceSet[choice=`0.0`]U=(1:`0.0`, 2:(beta(init=0) + (sigma(init=1) * bioDraws("V2", "NORMAL"))))av=(1:`1.0`, 2:`1.0`)))


The first function simply takes the log of the probability for each observation.

In [6]:
loglike = ll.loglikelihood(prob)
print(loglike)

log(MonteCarlo(exp(_bioLogLogitFullChoiceSet[choice=`0.0`]U=(1:`0.0`, 2:(beta(init=0) + (sigma(init=1) * bioDraws("V2", "NORMAL"))))av=(1:`1.0`, 2:`1.0`))))


The second function also involves the integral using Monte-Carlo simulation.

In [7]:
loglike = ll.mixedloglikelihood(condprob)
print(loglike)

log(MonteCarlo(exp(_bioLogLogitFullChoiceSet[choice=`0.0`]U=(1:`0.0`, 2:(beta(init=0) + (sigma(init=1) * bioDraws("V2", "NORMAL"))))av=(1:`1.0`, 2:`1.0`))))


Regression models are often used in the context of hybrid choice models. Consider the following model.

In [8]:
x = ex.Variable('x')
y = ex.Variable('y')
beta = ex.Beta('beta', 1, None, None, 0)
sigma = ex.Beta('sigma', 1, None, None, 0)
intercept = ex.Beta('intercept', 0, None, None, 0)
model = intercept + beta * x

The following function calculates the contribution to the likelihood. It is 

 \\[\frac{1}{\sigma} \phi\left( \frac{y-m}{\sigma} \right),\\]
 
 where $\phi(\cdot)$ is the pdf of the normal distribution.



In [9]:
like = ll.likelihoodregression(y, model, sigma)
print(like)

exp(((((-(((y - (intercept(init=0) + (beta(init=1) * x))) / sigma(init=1)) ** `2.0`)) / `2.0`) - (log((sigma(init=1) ** `2.0`)) / `2.0`)) - `0.9189385332`))


The following function calculates the log of the contribution to the likelihood. It is

\\[-\left( \frac{(y-m)^2}{2\sigma^2} \right) -
              \log(\sigma) - \frac{1}{2}\log(2\pi).\\]

In [10]:
loglike = ll.loglikelihoodregression(y, model, sigma)
print(loglike)

((((-(((y - (intercept(init=0) + (beta(init=1) * x))) / sigma(init=1)) ** `2.0`)) / `2.0`) - (log((sigma(init=1) ** `2.0`)) / `2.0`)) - `0.9189385332`)


We compare the two on a small database

In [11]:
df = pd.DataFrame({'x': [-2, -1, 0, 1, 2],
                   'y': [1, 1, 1, 1, 1]})
myData = db.Database('test', df)

In [12]:
lr = like.getValue_c(myData, prepareIds=True)
lr

array([0.00443185, 0.05399097, 0.24197073, 0.39894229, 0.24197073])

In [13]:
np.log(lr)

array([-5.41893852, -2.91893852, -1.41893852, -0.91893852, -1.41893852])

In [14]:
loglike.getValue_c(myData, prepareIds=True)

array([-5.41893852, -2.91893852, -1.41893852, -0.91893852, -1.41893852])